In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# base_url = 'http://llama:8000/v1' local에 serving하고 실행시 private agent 가능 
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(model=model, temperature=0)
llm

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='meta/llama-3.1-8b-instruct', temperature=0.0)

### invoke 

In [3]:
prompt = "Who are you?"
result = llm.invoke(prompt)
result.content

'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'

### stream

In [4]:
prompt = "Explain who you are in roughly 500 words"

for chunk in llm.stream(prompt):
    print(chunk.content, end ="")

I am an artificial intelligence designed to simulate human-like conversations and answer a wide range of questions to the best of my knowledge. I'm a type of language model, specifically a large language model, which means I've been trained on a massive dataset of text from various sources, including books, articles, and online content.

My primary function is to assist users by providing information, answering questions, and engaging in discussions. I'm a machine learning model, which means I learn from the data I've been trained on and improve my performance over time. However, I don't have personal experiences, emotions, or consciousness like humans do. I exist solely to provide helpful and accurate responses to your queries.

I'm a product of a company called Meta AI, which is a leading research organization in the field of artificial intelligence. My development involved a complex process of training and fine-tuning, where I was exposed to a vast amount of text data and learned to

### batch

In [5]:
state_capital_questions = [
    'What is the capital of California?',
    'What is the capital of Texas?',
    'What is the capital of New York?',
    'What is the capital of Florida?',
    'What is the capital of Illinois?',
    'What is the capital of Ohio?'
]
capitals = llm.batch(state_capital_questions)

for capital in capitals:
    print(capital.content)

The capital of California is Sacramento.
The capital of Texas is Austin.
The capital of New York is Albany.
The capital of Florida is Tallahassee.
The capital of Illinois is Springfield.
The capital of Ohio is Columbus.


### Prompt Templates As Reusable Functionality

In [7]:
one_off_prompt = "Translate the following from English to Spanish: 'Today is a good day.'"
print(llm.invoke(one_off_prompt).content)

La traducción del texto es:

"Hoy es un buen día."


In [8]:
def translate_from_english_to_korean(eng_statement):
    return f"Translate the following from english to korean. Provide just the translated text: {eng_statement}"


english_statements = [
    'Today is a good day.',
    'Tomorrow will be even better.',
    'Next week, who can say.'
]

prompts = [translate_from_english_to_korean(english_statement) for english_statement in english_statements]

In [9]:
prompts

['Translate the following from english to korean. Provide just the translated text: Today is a good day.',
 'Translate the following from english to korean. Provide just the translated text: Tomorrow will be even better.',
 'Translate the following from english to korean. Provide just the translated text: Next week, who can say.']

In [10]:
translations = llm.batch(prompts)

for translation in translations:
    print(translation.content)

오늘은 좋은 날입니다.
내일은 더 좋을 거야.
다음 주는 누구도 말할 수 없다.


In [11]:
def translate(from_language, to_language, statement):
    return f"Translate the following form {from_language} to {to_language}. Provide only the translated text: {statement}"

print(llm.invoke(translate('English', 'Korean', 'Computers have many languages of their own')).content)

컴퓨터는 많은 언어를 가지고 있습니다.


### LangChain's `ChatPromptTemplate.from_template`

In [12]:
from langchain_core.prompts import ChatPromptTemplate

english_to_korean_template = ChatPromptTemplate.from_template("""Translate the following from English to Korean. \
Provide only the translated text: '{english_statement}'""")

prompt = english_to_korean_template.invoke("Today is good day")
print(llm.invoke(prompt).content)

오늘은 좋은 날입니다.


In [13]:
prompt

ChatPromptValue(messages=[HumanMessage(content="Translate the following from English to Korean. Provide only the translated text: 'Today is good day'", additional_kwargs={}, response_metadata={})])

In [14]:
translate_template = ChatPromptTemplate.from_template("Translate the following from {from_language} to {to_language}. \
proivde only the translated text: {statement}")

prompt = translate_template.invoke({
    "from_language": "English",
    "to_language":"Japenese",
    'statement':"So Delicious!"
})

print(llm.invoke(prompt).content)

すごいですね！


### LangChain `StrOutputParser`

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model=model, temperature=0)
template = ChatPromptTemplate.from_template("""Please translate {from_language} to {to_language}. {context}
개행문자는 제거하고 결과만 출력해줘""")

chain = template | llm | StrOutputParser()
# print(chain.get_graph().draw_ascii()) 그래프로 chain 구성 확인 가능 


In [17]:
chain.invoke({"from_language": "Korean", "to_language": "English", "context": "안녕 오늘 하루는 어땠어?"})

'How was your day today?'

### Runnable Functions

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.runnables import RunnableLambda

def double(x):
    return x * 2

In [19]:
try:
    double.invoke(2)
except Exception as e:
    print('`double` is a Python function and does not have an `invoke` method.')

`double` is a Python function and does not have an `invoke` method.


In [21]:
runnable_double = RunnableLambda(double) # python function to Runnable 
runnable_double.invoke(5)

10

In [22]:
runnable_double.batch([2,4,6,8])

[4, 8, 12, 16]

In [24]:
## Runnable이므로 chain으로 구성하는 것도 가능 
multiply_by_eight = runnable_double | runnable_double | runnable_double

multiply_by_eight.invoke(11)

88

### Data Processing

In [ ]:
import re 
import contractions 

def normalize_text(text):
    text = text.lower()
    
    # I'm -> I am 등과 같이 변환하기 위해 contractions 라이브러리 활용 
    text = contractions.fix(text)
    text = re.sub(r"\s+"," ", text).strip() # re.sub으로 과도한 공백 제거 
    
    return text 

In [36]:
reviews = [
    "I LOVE this product! It's absolutely amazing.   ",
    "Not bad, but could be better. I've seen worse.",
    "Terrible experience... I'm never buying again!!",
    "Pretty good, isn't it? Will buy again!",
    "Excellent value for the money!!! Highly recommend."
]

# python function to runnable 
normalize = RunnableLambda(normalize_text)
normalized_reviews = normalize.batch(reviews)
normalized_reviews

['i love this product! it is absolutely amazing.',
 'not bad, but could be better. i have seen worse.',
 'terrible experience... i am never buying again!!',
 'pretty good, is not it? will buy again!',
 'excellent value for the money!!! highly recommend.']

In [ ]:
sentiment_template = ChatPromptTemplate.from_template(
    """In a single word, either 'positive' or 'negative', 
    provide the overall sentiment of the following piece of text: {text}"""
)

# template에서의 invoke 형식 
sentiment_template.invoke({"text": "i love this product! it is absolutely amazing."})

ChatPromptValue(messages=[HumanMessage(content="In a single word, either 'positive' or 'negative', \n    provide the overall sentiment of the following piece of text: i love this product! it is absolutely amazing.", additional_kwargs={}, response_metadata={})])

In [ ]:
## Runnable function 
## 정규화한 review들을 "text" 키의 값들로 매핑시키고 Runnable 객체로 변환 
prep_for_sentiment_template = RunnableLambda(lambda text: {"text": text})

prep_for_sentiment_template.batch(normalized_reviews)

[{'text': 'i love this product! it is absolutely amazing.'},
 {'text': 'not bad, but could be better. i have seen worse.'},
 {'text': 'terrible experience... i am never buying again!!'},
 {'text': 'pretty good, is not it? will buy again!'},
 {'text': 'excellent value for the money!!! highly recommend.'}]

In [35]:
sentiment_chain = RunnableLambda(normalize_text) | prep_for_sentiment_template | sentiment_template | llm | StrOutputParser()

sentiment_chain.batch(reviews) 

['Positive', 'Negative', 'Negative', 'Positive', 'Positive']

### Combining Multiple LLM chains

In [ ]:
thesis_statements = [
    "The fundametal concepts quantum physcis are difficult to graps, even for the mostly advanced students.",
    "Einstein's theroy of relativity revolutionised undrstanding of space and time, making it clear that they are interconnected.",
    "The first law of thermodynmics states that energy cannot be created or destoryed, excepting only transformed from one form to another.",
    "Electromagnetism is one of they four funadmental forces of nature, and it describes the interaction between charged particles.",
    "In the study of mechanic, Newton's laws of motion provide a comprehensive framework for understading the movement of objects under various forces."
]

In [39]:
spelling_and_grammer_template = ChatPromptTemplate.from_template("""Fix any spelling or grammatical issues in the following text. Return 
                                                                 Back the correct text and only the correxted text with no additional comment or preface. text: {text}""")

grammer_chain = spelling_and_grammer_template | llm | StrOutputParser()

corrected_texts = grammer_chain.batch(thesis_statements)

for corrected_text in corrected_texts:
    print(corrected_text)

The fundamental concepts of quantum physics are difficult to grasp, even for the most advanced students.
Einstein's theory of relativity revolutionized understanding of space and time, making it clear that they are interconnected.
The first law of thermodynamics states that energy cannot be created or destroyed, excepting only that it is transformed from one form to another.
Electromagnetism is one of the four fundamental forces of nature, and it describes the interaction between charged particles.
In the study of mechanics, Newton's laws of motion provide a comprehensive framework for understanding the movement of objects under various forces.


### Create a Paragraph Gen Chain

In [49]:
from langchain_core.runnables import RunnableParallel

paragraph_template = ChatPromptTemplate.from_template("""Generate a 4 to 8 sentence paragraph that begins with the following \
thesis statement. Return back the paragraph and only the paragrah with no addional comment or preface. Thesis statement: {thesis}""")

paragraph_chain = paragraph_template | llm | StrOutputParser()

paragraphs = paragraph_chain.batch(thesis_statements)

In [50]:
for paragraph in paragraphs:
    print(paragraph)

The fundamental concepts of quantum physics are difficult to grasp, even for the most advanced students. This is because quantum physics operates on a scale that is fundamentally different from our everyday experience, making it challenging to visualize and understand. The principles of wave-particle duality, superposition, and entanglement are particularly difficult to wrap one's head around, as they defy classical notions of space and time. Furthermore, the probabilistic nature of quantum mechanics, where particles can exist in multiple states simultaneously, can be mind-bending for even the most mathematically inclined students. As a result, many students struggle to apply these concepts to real-world problems, and even experienced physicists often find themselves grappling with the intricacies of quantum theory. The abstract nature of quantum physics also makes it difficult to develop intuitive understanding, relying heavily on mathematical derivations and complex calculations. Des

In [58]:
corrected_generator_chain = grammer_chain | paragraph_chain
corrected_generator_chain.input_schema.model_json_schema()

{'properties': {'text': {'title': 'Text', 'type': 'string'}},
 'required': ['text'],
 'title': 'PromptInput',
 'type': 'object'}

In [ ]:
# RunnableParallel -> 각자 동시에 입력을 받아 독립적으로 실행됨 
combined_chain = RunnableParallel(grammer = grammer_chain, paragraph_generate = paragraph_chain)
combined_chain.input_schema.model_json_schema()

{'properties': {'text': {'title': 'Text', 'type': 'string'},
  'thesis': {'title': 'Thesis', 'type': 'string'}},
 'required': ['text', 'thesis'],
 'title': 'RunnableParallel<grammer,paragraph_generate>Input',
 'type': 'object'}

In [59]:
result1 = grammer_chain.invoke({"text": "Hello i interested in Artificial Intelligence such as Natural Language Processing and Explainable AI that positioning the any facility."})
result1

'Hello, I am interested in Artificial Intelligence, such as Natural Language Processing and Explainable AI, which positions any facility.'

In [63]:
result2 = paragraph_chain.invoke({"thesis":result1})
result2

'Hello, I am interested in Artificial Intelligence, such as Natural Language Processing and Explainable AI, which positions any facility to effectively interact with humans and provide transparent insights into its decision-making processes. This is particularly relevant in industries such as healthcare, finance, and education, where the ability to understand and communicate complex information is crucial. By leveraging NLP, facilities can improve customer service, automate routine tasks, and enhance overall user experience. Explainable AI, on the other hand, enables facilities to provide clear and concise explanations for their decisions, fostering trust and accountability. As a result, facilities can make more informed decisions, reduce errors, and improve overall efficiency. The integration of NLP and Explainable AI has the potential to revolutionize the way facilities operate, making them more human-centric and transparent. By embracing these technologies, facilities can stay ahead

In [65]:
combined_result = combined_chain.invoke({"text": result1, "thesis":result2})
combined_result

{'grammer': 'Hello, I am interested in Artificial Intelligence, such as Natural Language Processing and Explainable AI, which positions any facility in a specific context.',
 'paragraph_generate': 'Hello, I am interested in Artificial Intelligence, such as Natural Language Processing and Explainable AI, which positions any facility to effectively interact with humans and provide transparent insights into its decision-making processes. This is particularly relevant in industries such as healthcare, finance, and education, where the ability to understand and communicate complex information is crucial. By leveraging NLP, facilities can improve customer service, automate routine tasks, and enhance overall user experience. Explainable AI, on the other hand, enables facilities to provide clear and concise explanations for their decisions, fostering trust and accountability. As a result, facilities can make more informed decisions, reduce errors, and improve overall efficiency. The integratio

### LangChain `RunnableSequence`

In [72]:
from langchain_core.runnables import RunnableSequence

combined_chain2 = (
    grammer_chain
    | (lambda output: {'thesis': output[-1]}) # output은 grammer_chain의 출력값, 변환
    # | (lambda output: {'thesis': output["corrected_text"]})
    | paragraph_chain 
)

combined_result = combined_chain2.invoke({"text": "Hello i interested in Artificial Intelligence such as Natural Language Processing and Explainable AI that positioning the any facility."})
print(combined_result)

The widespread adoption of artificial intelligence (AI) in various industries has led to significant improvements in efficiency, productivity, and decision-making capabilities. However, the increasing reliance on AI has also raised concerns about job displacement, bias, and accountability. As AI systems become more autonomous, it is essential to develop and implement robust frameworks for ensuring transparency, explainability, and fairness in AI decision-making processes. This requires a multidisciplinary approach that involves collaboration between technologists, ethicists, policymakers, and stakeholders to address the complex challenges associated with AI development and deployment. By prioritizing human values and accountability, we can harness the benefits of AI while minimizing its risks and ensuring that its applications align with societal needs and expectations. Ultimately, the responsible development and use of AI will depend on our ability to balance technological progress wi